### Task 1 (20% credit)

Assume you have a data set as below. It contains records of cars with three features: the type of the car (sports(1) or SUV(2)), the color of the car (red(1) or yellow(2)), and the origin of the car (domestic(1) or imported(2)). And the labels for the data are: stolen(1) and not(0). 

#### Questions:

a) Calculate the following sample probabilities:
P(Red|Stolen), P(SUV|Stolen), P(Domestic|Stolen), P(Red|Not Stolen) , P(SUV|Not Stolen), and P(Domestic|Not Stolen)

b) Suggest a classification for a red, domestic SUV - whether it will be stolen or not - using Naive Bayes classifier. 

Please perform all the necessary computations "by hands" rather than using python code.

In [2]:
import numpy as np
import pandas as pd
y=[1,0,1,0,1,0,1,0,0,1]
X=[[1,1,1,2,2,2,2,2,1,1],[1,1,1,1,1,2,2,2,2,1],[1,1,1,1,2,2,2,1,2,2]]
data=[y]+X
data=pd.DataFrame(data).T
data.columns=['Stolen?','Color','Type','Origin']

In [3]:
data

,Stolen?,Color,Type,Origin
0,1,1,1,1
1,0,1,1,1
2,1,1,1,1
3,0,2,1,1
4,1,2,1,2
5,0,2,2,2
6,1,2,2,2
7,0,2,2,1
8,0,1,2,2
9,1,1,1,2


a) The following sample probabilities will be:
$$
P(Red | Stolen)=\frac{P(Red ∩ Stolen)}{P(Stolen)} = \frac{3/10}{5/10} = \frac{3}{5}
$$
$$
P(SUV | Stolen)=\frac{P(SUV ∩ Stolen)}{P(Stolen)} = \frac{1/10}{5/10} = \frac{1}{5}
$$
$$
P(Domestic | Stolen)=\frac{P(Domestic ∩ Stolen)}{P(Stolen)} = \frac{2/10}{5/10} = \frac{2}{5}
$$
$$
P(Red | Not Stolen)=\frac{P(Red ∩ Not Stolen)}{P(Not Stolen)} = \frac{2/10}{5/10} = \frac{2}{5}
$$
$$
P(SUV | Not Stolen)=\frac{P(SUV ∩ Not Stolen)}{P(Not Stolen)} = \frac{3/10}{5/10} = \frac{3}{5}
$$
$$
P(Domestic | Not Stolen)=\frac{P(Domestic ∩ Not Stolen)}{P(Not Stolen)} = \frac{3/10}{5/10} = \frac{3}{5}
$$

b)The probability of a red domestic SUV that is stolen is:
$$
P(Red,Domestic,SUV|Stolen) \sim P(Red|Stolen)P(SUV|Stolen)P(Domestic|Stolen)P(Stolen)=3/5*1/5*2/5*1/2 = 1/125
$$
and the probability for Not Stolen
$$
P(Red,Domestic,SUV|NotStolen) \sim P(Red|NotStolen)P(SUV|NotStolen)P(Domestic|NotStolen)P(NotStolen)=2/5*3/5*3/5*1/2 = 9/125
$$

So the classifier will mark it "Not Stolen"

### Task 2 (25% credit)
Consider a following Guassian Naive Bayes problem.
We use eight factors to predict if people have diabetes or not. The variabls are:

y: The label (0 - no diabetes, 1 - diabetes)

t_pre: Number of times pregnant

glu: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

blood_p: Diastolic blood pressure (mm Hg)

triceps: Triceps skin fold thickness (mm)

serum: 2-Hour serum insulin (mu U/ml)

b_m: Body mass index (weight in kg/(height in m)^2)

pedigree_f: Diabetes pedigree function

age: Age (years)
#### ---------------------------------------------------------------------------------------------------------------------

#### Questions:

a)Train the classifier: use the training data to estimate prior probabilities $P(y=b)$ as well as the parameters (mean and standard deviation) of the sample distributions $P(x_i|y=b)$.

b)Perform the classification for the test sample. 

c)Compare your result to y_test and report the classification accuracy.

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import scipy
from scipy import stats
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
import statsmodels.formula.api as smf
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import random

Populating the interactive namespace from numpy and matplotlib


In [2]:
data_train=pd.read_csv("dia_train.csv") 
y_train=data_train.iloc[:,1] 
X_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("dia_test.csv")
y_test=data_test.iloc[:,1]
X_test=data_test.iloc[:,2:]

In [3]:
def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  #estimate priors
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  #estimate sample distribution paramters for p(xi|y=b)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

In [4]:
def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    #start from the priors
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu2'];
    #multiply by conditional probability densities p(xi|y=b)
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

In [5]:
train = pd.concat([y_train,X_train],axis=1)
test = pd.concat([y_test,X_test],axis=1)

a)classify training sample

In [6]:
train.head()

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
0,0,3,158,64,13,387,31.2,0.295,24
1,0,0,84,64,22,66,35.8,0.545,21
2,0,9,120,72,22,56,20.8,0.733,48
3,0,4,110,76,20,100,28.4,0.118,27
4,1,2,100,66,20,90,32.9,0.867,28


In [7]:
dp=trainNaiveBayes(train) #train the Gausian Naive Bayes Classifier
dp

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
mu1,0.6991525,2.690909,111.4667,69.20606,27.2,127.0061,31.70909,0.4686848,28.39394
sigma1,NaN,2.610061,24.69189,11.71329,10.43692,91.48614,6.337613,0.2917503,8.537362
mu2,0.3008475,4.070423,144.1408,74.56338,33.47887,209.2113,35.22394,0.6390423,35.78873
sigma2,NaN,3.51862,30.62646,13.79931,9.762697,126.921,6.258491,0.439042,10.26355


b)classify test set

In [8]:
test.head()

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
0,0,1,89,66,23,94,28.1,0.167,21
1,1,0,137,40,35,168,43.1,2.288,33
2,1,2,197,70,45,543,30.5,0.158,53
3,1,1,189,60,23,846,30.1,0.398,59
4,0,1,103,30,38,83,43.3,0.183,33


In [9]:
c_test=classifyNaiveBayes(test,dp) #run the classifier over the test sample
c_test

0      0
1      1
2      1
3      1
4      0
5      1
6      1
7      1
8      1
9      0
10     1
11     0
12     0
13     0
14     0
15     0
16     0
17     1
18     0
19     0
20     1
21     0
22     1
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
128    1
129    1
130    1
131    0
132    0
133    1
134    0
135    0
136    1
137    1
138    0
139    0
140    1
141    0
142    1
143    0
144    0
145    1
146    0
147    0
148    0
149    0
150    1
151    0
152    1
153    0
154    1
155    0
156    1
157    0
Name: y, dtype: float64

c)report accuracy

In [11]:
#accuracy over test sample
100.0*sum(c_test==test.y)/len(c_test)

79.11392405063292

In [12]:
#accuracy over training sample
Ct=classifyNaiveBayes(train,dp)
100.0*sum(Ct==train.y)/len(Ct)

76.27118644067797

### Task 3 (Credit 25%)
We have an artificial data set split, while the training set contains both - labeled (Label_train) and unlabeled (Unlabel) data. Column 'y' is the label, and columns '0','1','2' are categorical variables.

#### Questions:

a) Use the labeled part data_train to predict the labels of X_Label_test, and report the classification accuracy.

b) Improve the classification by using the unlabeled data data_Unlabel and the EM algorithm to predict labels of X_Label_test, and report the new accuracy by EM semi-supervised algorithm (use the same convergence criteria as in the lecture notebook). 

In [13]:
data_train=pd.read_csv("EM_train.csv")
y_Label_train=data_train.iloc[:,1] 
X_Label_train=data_train.iloc[:,2:] 

data_test=pd.read_csv("EM_test.csv")
y_Label_test=data_test.iloc[:,1]
X_Label_test=data_test.iloc[:,2:]

data_Unlabel=pd.read_csv("EM_Unlabel.csv")
X_Unlabel=data_Unlabel.iloc[:,1:]

In [14]:
train = pd.concat([y_Label_train,X_Label_train],axis=1)
test = pd.concat([y_Label_test,X_Label_test],axis=1)

In [18]:
def trainNaiveBayesDiscrete(trainData):
  #training discrete Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
  #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
  dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
  #split the training data between two labels
  ind1=tY==0
  ind2=tY==1
  #estimate P(y=b)  
  dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
  dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
  #estimate conditional probabilities P(x|y=b)
  for j in trainData.columns[1:]:
    for i in range(1,m+1):
        dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
        dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
  return dp

In [19]:
def classifyNaiveBayesDiscrete(classData,dp):
  #classifying using trained discrete Naive Bayes Classifier
  Y=classData[classData.columns[0]]*0 #initialize the empty array 
  for i in classData.index: #for al records to classify
    #start with the priors
    P1=dp[0][classData.columns[0]][1]; 
    P2=dp[1][classData.columns[0]][1];
    #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
    for j in classData.columns[1:]:
      P1=P1*dp[0][j][classData[j][i]]
      P2=P2*dp[1][j][classData[j][i]]
    Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
  return Y

In [21]:
dp=trainNaiveBayesDiscrete(train)
dp

[           y    0    1    2
 1  0.5555556    0  0.2    0
 2        NaN  0.2    0  0.2
 3        NaN  0.2    0    0
 4        NaN  0.2  0.2  0.2
 5        NaN    0    0    0
 6        NaN  0.4  0.6  0.6,            y     0     1     2
 1  0.4444444   0.5  0.25  0.25
 2        NaN  0.25   0.5   0.5
 3        NaN     0  0.25     0
 4        NaN     0     0     0
 5        NaN  0.25     0     0
 6        NaN     0     0  0.25]

In [22]:
C=classifyNaiveBayesDiscrete(test,dp)

In [24]:
accuracy=format(100.0*sum(C==y_Label_test)/len(y_Label_test))
print "We correctly classified {0} percents of the trips based on the labeled data only".format(accuracy)     

We correctly classified 63.8888888889 percents of the trips based on the labeled data only


In [25]:
#implementation of Expectation-Maximization algorithm for partially labeled data
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    
    if (sum(sum((dp1[0]-dp[0])**2))+sum(sum((dp1[1]-dp[1])**2)))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels:    
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
            L=L+math.log(P)
        
        print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
  return dp

In [26]:
#perform EM estimation for theta
dpEM=EM(X_Label_train,y_Label_train,X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(test,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C == y_Label_test)/len(y_Label_test)
print "After EM we correctly classified {0} percents of the trips".format(acc)

Iteration 1: log maximum liklihood = -590.452072842
Iteration 2: log maximum liklihood = -572.346722303
Iteration 3: log maximum liklihood = -559.190769042
Iteration 4: log maximum liklihood = -556.633109435
Iteration 5: log maximum liklihood = -556.067798435
After EM we correctly classified 94.4444444444 percents of the trips


### Task 4 (Credit 30%)
For the similar artifitial data uploaded below:

#### Question: 

a) Apply the EM algorithm (no observed labels, random initial choice of $\theta$) for clustering the data records into two clusters. Report your result (a vector of cluster numbers for each data record). 

b) Repeat the clustering 10 times with different random choices of $\theta$ and analyze the stability of the clustering (matching labels accross different clusterings (use the choice of 0 and 1 labels best matching the previous clustering), estimate average label and its standard error for each record).

a)

In [32]:
data=pd.read_csv("EM_Cluster.csv")
X=data.iloc[:,1:]
y=data.iloc[:,1]

In [40]:
np.random.seed(2222)
#initialize theta randomly
dp[0].iloc[0,0]=np.random.uniform(0,1)
dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
for j in range(1,len(dp[0].T)):
    b=np.random.uniform(0,1,len(dp[0]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[0].iloc[:,j]=b
for j in range(1,len(dp[1].T)):
    b=np.random.uniform(0,1,len(dp[1]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[1].iloc[:,j]=b


In [41]:
dp

[           y         0         1         2
 1  0.7272002  0.080634  0.268894  0.234451
 2        NaN  0.237055  0.311129  0.118511
 3        NaN  0.202867  0.081060  0.214199
 4        NaN  0.058439  0.012051  0.010802
 5        NaN  0.175873  0.121096  0.139406
 6        NaN  0.245132  0.205770  0.282631,
            y         0         1         2
 1  0.2727998  0.255090  0.114639  0.195064
 2        NaN  0.104158  0.180371  0.168233
 3        NaN  0.235106  0.266673  0.177792
 4        NaN  0.081513  0.046983  0.221156
 5        NaN  0.153593  0.204850  0.120076
 6        NaN  0.170539  0.186485  0.117679]

In [42]:
#perform EM estimation for theta
dpEM=EM([],[],X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(X,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y)/len(y)
if acc<50: #if most of the labels are misclassified it is just the matter of swapping the labels 1<->0
    acc=100-acc
print "After EM we correctly classified {0} percents of the trips".format(acc)

Iteration 1: log maximum liklihood = -553.658100684
Iteration 2: log maximum liklihood = -551.463470237
Iteration 3: log maximum liklihood = -549.717415096
Iteration 4: log maximum liklihood = -547.945947925
Iteration 5: log maximum liklihood = -545.369863538
Iteration 6: log maximum liklihood = -540.421541426
Iteration 7: log maximum liklihood = -531.356990769
Iteration 8: log maximum liklihood = -520.319563354
Iteration 9: log maximum liklihood = -513.905882065
Iteration 10: log maximum liklihood = -510.608612293
Iteration 11: log maximum liklihood = -507.707083252
Iteration 12: log maximum liklihood = -505.764678159
After EM we correctly classified 97.1428571429 percents of the trips


b)
when i choose a large number of $\theta$, such as 8888, the accuracy goes to 100 percent